#### **Qualità dei risultati**

In [134]:
import re
import pandas as pd
import copy
import math

#### Mpileup

In [ ]:

file1 = open("mpileup_V5.txt",'r')
file2 = open("mpileup_samtools.txt",'r')

file1_lines = file1.readlines()
file2_lines = file2.readlines()

with open('V5_adj.csv', 'w') as f1,open('samtools_adj.csv', 'w') as f2:

    len_file1=math.ceil(len(file1_lines))
    len_file2=math.ceil(len(file2_lines))
    
    for i in range(len_file1):
        listed1=re.sub(' +', ',',file1_lines[i])
        if listed1.count(',')==5:
            f1.write(listed1)

    for i in range(len_file2):
        listed2=re.sub('\t', ',',file2_lines[i])
        if listed2.count(',')==5:
            f2.write(listed2)

file1.close()
file2.close()


In [ ]:

df_1=pd.read_csv('V5_adj.csv',header=None,names=['rname1','pos1','N1','num1','seq1','qual1'])
df_2=pd.read_csv('samtools_adj.csv',header=None,names=['rname2','pos2','N2','num2','seq2','qual2'])
df_result = pd.merge(df_1, df_2, how="outer",left_on='pos1', right_on='pos2')
df_result_excluded=df_result[(df_result.pos1.isnull())|(df_result.pos2.isnull()) ]
df_result_final=df_result.drop(df_result_excluded.index)

In [ ]:
df_result_final.to_csv('df_result_final.csv',index=False)

In [43]:
df_result_final=pd.read_csv('df_result_final.csv')
max(df_result_final.pos1)
correct=0
for index, row in df_result_final.iterrows():
    compare1=''.join(sorted(row[['num1','seq1','qual1']].apply(lambda x: ''.join(sorted(str(x))))))
    compare2=''.join(sorted(row[['num2','seq2','qual2']].apply(lambda x: ''.join(sorted(str(x))))))

    if compare1==compare2:
        correct+=1
    else:
        len1=len(compare1)
        len2=len(compare2)
        if len1 < len2:
            len_diff=len2-len1
            compare1=compare1 + ' '*len_diff
        if len1>len2:
            len_diff=len1-len2
            compare2=compare2 + ' '*len_diff
        else:
            pass
        
        difference1 = ''.join(x for x, y in zip(list(compare1), list(compare2)) if x != y)
        difference2 = ''.join(y for x, y in zip(list(compare1), list(compare2)) if x != y)

        print('index:',index,'\tdifference:',difference1,difference2,'\tcompare:',compare1,compare2)

print('\noverall correctness: {}'.format(correct/len(df_result_final)))

In [ ]:
#it has been created a program executing the above commands

!python3 data_quality/mpileup/mpileup_quality.py

#### Consensus

In [ ]:
#execute the python script
#it results the scores for which the mean and variance will be performed

!python3 data_quality/consensus/consensus_quality.py

#### coverage

In [ ]:
#the following command will output the file coverage_output.txt

!python3 data_quality/coverage/coverage_quality.py

In [143]:
import pandas as pd
coverage=pd.read_csv('coverage/coverage_output.txt',header=None)
coverage=coverage.rename(columns={0:'startpos',1:'endpos',2:'numreads',3:'covbases',4:'coverage',5:'meandepth',6:'meanbaseq',7:'meanmapq'})

In [144]:
coverage

,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,meanmapq
0,16573063,16593063,593,10678,53.387331,1.466477,39.421183,36.898820
1,16573063,16593063,593,10675,53.372300,1.480330,39.500000,36.900000
2,26486163,26506163,1067,14795,73.971301,2.638118,39.388357,36.966261
3,26486163,26506163,1067,14795,73.971300,2.663320,39.500000,37.000000
4,23289397,23309397,1070,14180,70.896455,2.631668,38.928466,36.897196
...,...,...,...,...,...,...,...,...
195,17957685,17977685,802,13528,67.636600,2.003100,39.400000,36.800000
196,14413784,14433784,935,14444,72.216389,2.308785,39.282952,36.974332
197,14413784,14433784,935,14444,72.216400,2.331580,39.400000,37.000000
198,214599,234599,291,7573,37.863107,0.727514,39.616194,0.752577


In [145]:
import numpy as np
coverage_diff_covbases=coverage.groupby('startpos').agg({'coverage': 'diff' })
coverage_diff_covbases=pd.DataFrame({'diff':list(coverage_diff_covbases[coverage_diff_covbases.coverage.notna()]['coverage']),'max':list(coverage.groupby('startpos').agg({'coverage': 'max' })['coverage'])})

coverage_diff_coverage=coverage.groupby('startpos').agg({'meandepth': 'diff'})
coverage_diff_coverage=pd.DataFrame({'diff':list(coverage_diff_coverage[coverage_diff_coverage.meandepth.notna()]['meandepth']),'max':list(coverage.groupby('startpos').agg({'meandepth': 'max' })['meandepth'])})

coverage_diff_meandepth=coverage.groupby('startpos').agg({'meanbaseq': 'diff'})
coverage_diff_meandepth=pd.DataFrame({'diff':list(coverage_diff_meandepth[coverage_diff_meandepth.meanbaseq.notna()]['meanbaseq']),'max':list(coverage.groupby('startpos').agg({'meanbaseq': 'max' })['meanbaseq'])})

coverage_diff_meanbaseq=coverage.groupby('startpos').agg({'meanmapq': 'diff'})
coverage_diff_meanbaseq=pd.DataFrame({'diff':list(coverage_diff_meanbaseq[coverage_diff_meanbaseq.meanmapq.notna()]['meanmapq']),'max':list(coverage.groupby('startpos').agg({'meanmapq': 'max' })['meanmapq'])})



In [127]:
coverage_diff_covbases['perc']=coverage_diff_covbases['diff']/coverage_diff_covbases['max']
coverage_diff_coverage['perc']=coverage_diff_coverage['diff']/coverage_diff_coverage['max']
coverage_diff_meandepth['perc']=coverage_diff_meandepth['diff']/coverage_diff_meandepth['max']
coverage_diff_meanbaseq['perc']=coverage_diff_meanbaseq['diff']/coverage_diff_meanbaseq['max']

In [130]:
coverage_diff_covbases=coverage_diff_covbases.abs()
coverage_diff_coverage=coverage_diff_coverage.abs()
coverage_diff_meandepth=coverage_diff_meandepth.abs()
coverage_diff_meanbaseq=coverage_diff_meanbaseq.abs()

In [132]:
covbases_mean=coverage_diff_covbases['perc'].mean()
covbases_var=coverage_diff_covbases['perc'].var()
coverage_mean=coverage_diff_coverage['perc'].mean()
coverage_var=coverage_diff_coverage['perc'].var()
meandepth_mean=coverage_diff_meandepth['perc'].mean()
meandepth_var=coverage_diff_meandepth['perc'].var()
meanbaseq_mean=coverage_diff_meanbaseq['perc'].mean()
meanbaseq_var=coverage_diff_meanbaseq['perc'].var()

In [146]:
print(covbases_mean)
print(covbases_var)
print(coverage_mean)
print(coverage_var)
print(meandepth_mean)
print(meandepth_var)
print(meanbaseq_mean)
print(meanbaseq_var)

6.177360917584786e-05
1.3599198588317813e-08
0.018989231126132173
0.0011189356661424784
0.0030556561866727718
8.017964166284337e-07
0.0011760936669560412
6.046832103545184e-06
